# 2: Train XGBoost Model

Author: Daniel Lusk

## Imports and configuration

In [57]:
import xgboost as xgb
from TrainModelConfig import TrainModelConfig
from utils.data_retrieval import all_gdfs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

config = TrainModelConfig()


def print_shapes(X, Y, rows_dropped=True) -> None:
    print("X shape:", X.shape)
    print("Y shape:", Y.shape)
    
    if rows_dropped:
        rows = abs(Y.shape[0] - X.shape[0])
        print("Rows dropped:", rows)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [2]:
X_fns = config.WC_fns + config.MODIS_fns + config.soil_fns
Y_fns = config.iNat_fns

X = all_gdfs(X_fns)
Y = all_gdfs(Y_fns)

Compute Preciptation Annual Range by subtracting BIO14 from BIO13

In [3]:
bio_13 = X.loc[:, ["bio_13" in x for x in X.columns]].values
bio_14 = X.loc[:, ["bio_14" in x for x in X.columns]].values
X["wc2.1_10m_bio_13-14"] = bio_13 - bio_14

Drop the unnecessary `x`, `y`, `band` and `spatial_ref` columns.

In [4]:
X = X.drop(columns=["x", "y", "band", "spatial_ref"])
Y = Y.drop(columns=["x", "y", "band", "spatial_ref"])

## XGBoost

To-Dos:

1. Match predictors with response variable(s) (just use one variable at first to test all the folllowing steps)
2. Remove all-null predictors/response vars
3. ~~Standardize the data~~ Not actually necessary for tree-based models!
4. Divide into spatial CV splits
5. Exclude location columns
6. Test out training
7. Identify optimal hyperparams with grid search + spatial CV
8. Repeat training, but remove ANY rows with NA

Use just one response variable while developing the methodology. In this case, use specific leaf area (SLA).

In [8]:
Y = Y[["geometry", "iNat_SLA_05deg_ln"]]

# Drop response variable NAs
Y = Y.dropna()

Y.head(5)

,y,x,geometry,iNat_SLA_05deg_ln
0,89.75,-179.75,POINT (-179.75000 89.75000),NaN
1,89.75,-179.25,POINT (-179.25000 89.75000),NaN
2,89.75,-178.75,POINT (-178.75000 89.75000),NaN
3,89.75,-178.25,POINT (-178.25000 89.75000),NaN
4,89.75,-177.75,POINT (-177.75000 89.75000),NaN


<p>1. Match predictors with response variable</p>

In [42]:
X = X.loc[X["geometry"].isin(Y["geometry"])]
print_shapes(X, Y)

X shape: (23178, 132)
Y shape: (23178, 4)


<p>2. Remove all-NA predictors</p>

In [58]:
X = X.dropna(subset=X.columns.difference(["geometry"]), how="all")
print_shapes(X, Y)

X shape: (23166, 130)
Y shape: (23178, 2)
Rows dropped: 12


<p>3. Standardize the data by centering to the mean and scaling to the STD (skipped because not actually necessary for tree-based models)</p>

<p>4. Divide into geographic splits for spatial K-fold cross-validation</p>

In [64]:
import matplotlib.pyplot as plt
import spacv

XYs = X["geometry"]
skcv = spacv.SKCV().split(XYs)


ImportError: cannot import name 'asPolygon' from 'shapely.geometry' (/home/lusk/mambaforge/envs/traits/lib/python3.9/site-packages/shapely/geometry/__init__.py)

## Old

Drop NaNs from labels and convert dataframes to numpy arrays

In [ ]:
X_np = X.to_numpy()
Y_np = Y.dropna().to_numpy()

Split into train and test and convert data into DMatrices for XGBoost

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, test_size=2.0)

D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)